# 1. Setup Google Drive and Environment


In [ ]:
%pip install transformers datasets evaluate sacrebleu
import torch
import numpy as np
import pandas as pd
from typing import Dict
import torch
from datasets import load_dataset, load_from_disk
from transformers import DataCollatorWithPadding
import evaluate 

from transformers import (
    AutoConfig,
    AutoModelForSequenceClassification,
    AutoTokenizer,
    EvalPrediction,
    Trainer,
    TrainingArguments,
    set_seed,
)


from sklearn.model_selection import train_test_split
from datasets import Dataset



from transformers import AutoModel
import torch.nn as nn
import torch.nn.functional as F



# 2. Model and Hyper-Parameters


In [ ]:
# Model: Distil-Bert
language_model_name = "google/mt5-small"

# HYPER-PARAMETERS

batch_size = 32
learning_rate = 1e-4
weight_decay = 0.001
epochs = 4
device = "cuda" if torch.cuda.is_available() else "cpu"
set_seed(42)


# 3. Accessing the dataset

In [ ]:
dataset = pd.read_csv("NonModernItDataset.csv")
dataset = Dataset.from_pandas(dataset)
dataset = dataset.train_test_split(0.2,shuffle=True)

dataset


# 4. Function to compute the Metrics


# 5. Initialization of the model


In [ ]:
from transformers import AutoTokenizer

checkpoint = "google-t5/t5-small"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
source_lang = "source"
target_lang = "target"
prefix = "Translate Not Modern Itailan to Modern Italian: "

def preprocess_function(examples):
    inputs = [prefix + example for example in examples[source_lang]]
    #print(inputs)
    targets = [example for example in examples[target_lang]]
    model_inputs = tokenizer(inputs, text_target=targets, max_length=128, truncation=True)
    return model_inputs

dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
source_lang = "source"
target_lang = "target"
prefix = "Translate Not Modern Itailan to Modern Italian: "
metric = evaluate.load("sacrebleu")



def postprocess_text(preds, labels):
    preds = [pred.strip() for pred in preds]
    labels = [[label.strip()] for label in labels]

    return preds, labels


def compute_metrics(eval_preds):
    preds, labels = eval_preds
    if isinstance(preds, tuple):
        preds = preds[0]
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)

    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)

    result = metric.compute(predictions=decoded_preds, references=decoded_labels)
    result = {"bleu": result["score"]}

    prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result


In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)

In [ ]:
from transformers import AutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer

model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint)

# 8. Trainer

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="non-modern-it-model",
    eval_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=10,
    predict_with_generate=True,
    fp16=True, #change to bf16=True for XPU
    push_to_hub=False,
    report_to="none",

)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["test"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

In [ ]:
# Let's Train ...
trainer.train()

# 9. Evaluating the performance of our model

In [ ]:
# Evaluate the model ...
trainer.evaluate()

In [ ]:
text = prefix + "Li occhi porta la mia donna sovra lor gloria."
text

In [ ]:
from transformers import AutoTokenizer

inputs = tokenizer(text, return_tensors="pt").input_ids

In [ ]:
from transformers import AutoModelForSeq2SeqLM

outputs = model.to("cpu").generate(inputs, max_new_tokens=40, do_sample=True, top_k=30, top_p=0.95)

In [ ]:
tokenizer.decode(outputs[0], skip_special_tokens=True)